# ML Lifecycle Management

> _"Hardest Part of ML isn’t ML, it’s Data"_

![Hardest part of ML](images/mlflow_tech_icon.png)

[Hidden Technical Debt in Machine Learning Systems](http://papers.neurips.cc/paper/5656-hidden-technical-debt-in-machine-learning-systems.pdf) (Google NIPS, 2015)  

## Objective

This module is going to introduce you to [MLflow](https://mlflow.org/docs/latest/index.html) for machine learning lifecycle management. We will introduce you to MLflow for... 

1. managing model experiments,
1. tracking hyperparameter tuning,
1. registering and serving models.

MLflow is a very comprehensive tool so we are just going to touch the surface. However, we will provide you with resources to dig deeper into MLflow.

## Problem

The ML process can be tedious, difficult, and result in lots of technical debt.

* How do we ***track***...
  - model runs
  - hyperparameter experimentations
  - performance metrics
  
* How do we manage ML ***projects*** for...
  - reproducibility
  - collaboration
  
* How do we package ML ***models*** for downstream deployment

* How do we ***register*** models for managing...
  - model lineage
  - model versioning
  - stage transitions (i.e. dev to stage to prod)

## Intro to MLflow

MLflow is an open source platform designed to manage the complete Machine Learning Lifecycle.

![](images/mlflow_capabilities.png)

* Used heavily --> 1.7M+ monthly downloads
* Well supported --> 170+ contributors & 40 contributing organizations
* Well documented
   - [mlflow.org](https://mlflow.org/)
   - [github.com/mlflow](https://github.com/mlflow)
   - [Slack channel](https://join.slack.com/t/mlflow-users/shared_invite/zt-g6qwro5u-odM7pRnZxNX_w56mcsHp8g)
   - [stackoverflow.com/questions/tagged/mlflow](https://stackoverflow.com/questions/tagged/mlflow)
   - [twitter.com/MLflow](https://twitter.com/MLflow)
   - [databricks.com/mlflow](https://databricks.com/product/managed-mlflow)

## Model Tracking

Key concepts in tracking:

* __Date/time:__ Start and end time of each model run.
* __Paramaters:__ Key-value inputs to your code.
* __Metrics:__ Numeric values to track how your model’s loss function is converging.
* __Artifacts:__ Output files in any format, including models.
* __Source:__ what code ran?

There are several ways to record your modeling experiment:

- SQLAlchemy compatible database
- remotely to a tracking server
- __locally__

Let's create an experiment:

In [1]:
import mlflow

mlflow.set_experiment("Predicting income")

<Experiment: artifact_location='file:///Users/bradley.boehmke/Desktop/workspace/trainings/advanced-python-datasci/notebooks/mlruns/1', experiment_id='1', lifecycle_stage='active', name='Predicting income', tags={}>

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Tip</b></p>
    <p class="last"><tt class="docutils literal">set_experiment</tt> will create and set an experiment if the experiment does not already exist.</p>
</ul>
</div>

Note the new local `mlruns/` directory:
    
![mlruns directory](images/mlruns_directory.png)

Before we record a model run let's import and prepare our data:

In [2]:
# packages used
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# import data
adult_census = pd.read_csv('../data/adult-census.csv')

# separate feature & target data
target = adult_census['class']
features = adult_census.drop(columns='class')

# drop the duplicated column `"education-num"` as stated in the data exploration notebook
features = features.drop(columns='education-num')

# split into train & test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=123)

# create selector object based on data type
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

# get columns of interest
numerical_columns = numerical_columns_selector(features)
categorical_columns = categorical_columns_selector(features)

# preprocessors to handle numeric and categorical features
numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)
])

To log a model run we use `start_run()` along with various other logging functions:

- `log_param(s)`: to log parameters of interest
- `log_metrics(s)`: to log model metrics of interest
- `set_tag(s)`: to log decsriptive information (version number, platform ran on, etc.)
- `log_artifact(s)`: allows you to log items such as data, models, files, etc.

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline


mlflow.start_run()

mlflow.log_param('max_iter', 500)
model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

_ = model.fit(X_train, y_train)

accuracy = model.score(X_test, y_test)
mlflow.log_metric('accuracy', accuracy)

mlflow.end_run()

A more common approach you'll see is to use `start_run()` as a [context manager](https://book.pythontips.com/en/latest/context_managers.html).

In [4]:
with mlflow.start_run() as run:

    mlflow.log_param('max_iter', 500)
    model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

    _ = model.fit(X_train, y_train)

    accuracy = model.score(X_test, y_test)
    mlflow.log_metric('accuracy', accuracy)

<div class="admonition tip alert alert-warning">
    <p class="first admonition-title" style="font-weight: bold;"><b>Question?</b></p>
<p class="last">
What other useful information could we log?
</p>
</div>

The following also logs the model type and the model itself as an artifact

In [5]:
with mlflow.start_run() as run:

    mlflow.set_tag('Estimator', 'LogisticRegression')
    mlflow.log_param('max_iter', 500)
    model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

    _ = model.fit(X_train, y_train)
    mlflow.sklearn.log_model(model, 'baseline_model')

    accuracy = model.score(X_test, y_test)
    mlflow.log_metric('accuracy', accuracy)

## MLflow UI

We can programmatically retrieve our model run information but MLflow also provides a very nice UI that displays information.

- Remove the `#` from the following line of code
- Click on the local URL provided

![Launch MLflow UI](images/local_url.png)

In [10]:
#!mlflow ui

<div class="admonition warning alert alert-danger">
    <p class="first admonition-title" style="font-weight: bold;"><b>Warning</b></p>
<p class="last">You'll need to stop the previous code cell when you are done viewing the MLflow UI.
</p>
</div>

## Auto logging

MLflow has built-in [auto logging](https://www.mlflow.org/docs/latest/tracking.html#automatic-logging) for many common model libraries:

- Scikit-learn
- TensorFlow & Keras
- XGBoost
- Spark ML
- Pytorch
- etc.

This can simplify our logging.

For example, the built in `sklearn.autolog` functionality will automatically log:

- Training score obtained by `estimator.score`
- Parameters obtained by `estimator.get_params`
- Model class name
- Fitted estimator as an artifact

In [8]:
# enable autologging
mlflow.sklearn.autolog()

with mlflow.start_run() as run:

    model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))
    _ = model.fit(X_train, y_train)

    mlflow.log_metric('test_accuracy', model.score(X_test, y_test))

2021/12/27 10:02:19 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('columntransformer', ColumnTransformer(transformers=[('one-hot-encoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['workclass', 'education', 'marital-status',
                     ...`
2021/12/27 10:02:19 WARNING mlflow.utils: Truncated the value of the key `columntransformer`. Truncated value: `ColumnTransformer(transformers=[('one-hot-encoder',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 ['workclass', 'education', 'marital-status',
                                  'occupatio...`
2021/12/27 10:02:19 WARNING mlflow.utils: Truncated the value of the key `columntransformer__transformers`. Truncated value: `[('one-hot-encoder', OneHotEncoder(handle_unknown='ignore'), ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native

Let's check out this auto-logged run in the UI. You'll notice some additional information logged.

In [11]:
#!mlflow ui

## Hyperparameter tuning

So far, we've just been logging individual runs.

However, MLflow makes it easy to track hyperparameter search experiments:

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

# basic model object
knn = KNeighborsClassifier()

# Create grid of hyperparameter values
hyper_grid = {'knn__n_neighbors': [5, 10, 15, 20]}

# create preprocessor & modeling pipeline
pipeline = Pipeline([('preprocessor', preprocessor), ('knn', knn)])

# enable autologging
mlflow.sklearn.autolog()

with mlflow.start_run() as run:
    # Tune a knn model using grid search
    grid_search = GridSearchCV(pipeline, hyper_grid, cv=5, scoring='roc_auc', n_jobs=-1)
    results = grid_search.fit(X_train, y_train)

2021/12/27 10:24:33 WARNING mlflow.utils: Truncated the value of the key `estimator`. Truncated value: `Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['wor...`
2021/12/27 10:29:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/bradley.boehmke/Downloads/ENTER/envs/uc-python/lib/python3.8/site-packages/mlflow/models/signature.py:129: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can de

If we look at the MLflow UI we'll notice that autologging a parameter search results in a single parent run and nested child runs, which contains:

* Parent
   - Training score
   - Best parameter combination
   - Fitted best estimator
   - and more
* Child
   - CV test score for each parameter combination

In [19]:
#!mlflow ui

## Registering models

MLflow provides a ___Model Registry___ that provides a centralized and collaborative approach to model lifecycle management.

__One Collaborative Hub__: The Model Registry provides a central hub for making models discoverable, improving collaboration and knowledge sharing across the organization.

![](images/registered_models.png)

__Manage the entire Model Lifecycle (MLOps)__: The Model Registry provides lifecycle management for models from experimentation to deployment, improving reliability and robustness of the model deployment process.

1. Overview of active model versions and their deployment stage
2. Request/Approval workflow for transitioning deployment stages

![](images/model_registry_mlops.png)

__Visibility and Governance__: The Model Registry provides full visibility into the deployment stage of all models, who requested and approved changes, allowing for full governance and auditability.

1. Full activity log of stage transition requests, approvals, etc.

![](images/model_registry_visibility.png)

Full provenance from Model marked production in the Registry to ...
1. Run that produced the model
2. Notebook that produced the run
3. Exact revision history of the notebook that produced the run

![](images/model_registry_governance.png)

Let's go ahead and register a model

<div class="admonition note alert alert-info">
    <p class="first admonition-title" style="font-weight: bold;"><b>Note</b></p>
<p class="last">Although in this example we register a model via the MLflow UI, we can also register a model programmatically.</p>
</div>

In [7]:
!mlflow server \
    --backend-store-uri sqlite:///mlflow.db \
    --default-artifact-root ./artifacts \
    --host 127.0.0.1

[2021-12-28 15:47:56 -0500] [56706] [INFO] Starting gunicorn 20.1.0
[2021-12-28 15:47:56 -0500] [56706] [INFO] Listening at: http://127.0.0.1:5000 (56706)
[2021-12-28 15:47:56 -0500] [56706] [INFO] Using worker: sync
[2021-12-28 15:47:56 -0500] [56708] [INFO] Booting worker with pid: 56708
[2021-12-28 15:47:56 -0500] [56709] [INFO] Booting worker with pid: 56709
[2021-12-28 15:47:56 -0500] [56710] [INFO] Booting worker with pid: 56710
[2021-12-28 15:47:56 -0500] [56711] [INFO] Booting worker with pid: 56711
^C
[2021-12-28 15:48:35 -0500] [56706] [INFO] Handling signal: int
[2021-12-28 15:48:35 -0500] [56711] [INFO] Worker exiting (pid: 56711)


In [2]:
remote_server_uri = "http://127.0.0.1:5000" # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)

In [ ]:
!mlflow ui

[2021-12-28 15:49:14 -0500] [56857] [INFO] Starting gunicorn 20.1.0
[2021-12-28 15:49:14 -0500] [56857] [INFO] Listening at: http://127.0.0.1:5000 (56857)
[2021-12-28 15:49:14 -0500] [56857] [INFO] Using worker: sync
[2021-12-28 15:49:14 -0500] [56860] [INFO] Booting worker with pid: 56860


## Wrapping up